In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from tensorflow.keras.models import load_model

face_detector = YOLO("yolov8n-face.pt")

gender_model = load_model("gender_classification_model.h5")
gender_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
classes = ["Male", "Female", "Other"]

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detector.predict(rgb_frame, verbose=False)
    faces = results[0].boxes.xyxy.cpu().numpy()

    male_count, female_count = 0, 0

    for box in faces:
        x1, y1, x2, y2 = map(int, box)
        face_roi = frame[y1:y2, x1:x2]

        if face_roi.shape[0] > 0 and face_roi.shape[1] > 0:
            face_resized = cv2.resize(face_roi, (64, 64)).astype(np.float32) / 255.0
            face_resized = np.expand_dims(face_resized, axis=0)

            prediction = gender_model.predict(face_resized, verbose=0)
            label = classes[np.argmax(prediction)]

            if label == "Male":
                male_count += 1
                color = (255, 0, 0)
            else:
                female_count += 1
                color = (0, 0, 255)

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    cv2.putText(frame, f"Male: {male_count} | Female: {female_count}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 
                0.8, (0, 255, 0), 2)

    cv2.imshow("Multi-Face Gender Classification", frame)

    key = cv2.waitKey(10) & 0xFF
    if key == ord('q'):
        print("Exiting...")
        break

cap.release()
cv2.destroyAllWindows()

Exiting...


In [2]:
#import requests

#url = "https://github.com/akanametov/yolo-face/releases/download/v0.0.0/yolov8n-face.pt"
#filename = "yolov8n-face.pt"

#response = requests.get(url, stream=True)
#with open(filename, "wb") as file:
#    for chunk in response.iter_contentchunk_size=1024):
#        file.write(chunk)
(
#print("Download completed: yolov8n-face.pt")

SyntaxError: unexpected EOF while parsing (3074671048.py, line 11)